In [1]:
import pandas as pd
import pandas_profiling as pp
import dtale
import seaborn as sns
from questionnaires_aggregation import c_ssrs_intake_aggregation
from utils import impute_from_column, simple_eda
from consts.questions_columns import c_ssrs_intake
from consts.questions_text_mapping import intake_c_ssrs_rename


from sklearn.decomposition import PCA, KernelPCA, NMF
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from umap import UMAP


Using TensorFlow backend.


In [2]:
df = pd.read_csv(r'../data_for_research.csv', na_values = ' ')
df = df[df.redcap_event_name == 'intake_arm_1']

name = 'C_SSRS_intake'
questions_text = False
activate_dtale = False
dim_reduction = UMAP

In [3]:
df['c_ssrs_intake_missing_rate'] = df[c_ssrs_intake].isna().sum(axis=1) / len(c_ssrs_intake)

columns = c_ssrs_intake  + ['c_ssrs_intake_missing_rate']

In [4]:
for time in ['life', '2weeks', 'recent']:
    for severity in ['stb', 'idea']:
        agg_column = f'c_ssrs_intake_{time}_{severity}'
        df[agg_column] = c_ssrs_intake_aggregation(df, severity = severity, time = time)
        columns.append(agg_column)

In [5]:
if questions_text:
    simple_eda(df, columns = columns, title = name, path = '../pandas_profiling', rename=intake_c_ssrs_rename)
else:
    simple_eda(df, columns = columns, title = name, path = '../pandas_profiling',)

C:\Users\nogur\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\nogur\anaconda3\lib\site-packages\pandas_profiling\model\missing.py:89: UserWarning:

There was an attempt to generate the Matrix missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(missing_diagrams={"Matrix": False}`)
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'keyword grid_b is not recognized; valid keywords are ['size', 'width', 'color', 'tickdir', 'pad', 'labelsize', 'labelcolor', 'zorder', 'gridOn', 'tick1On', 'tick2On', 'label1On', 'label2On', 'length', 'direction', 'left', 'bottom', 'right', 'top', 'labelleft', 'labelbottom', 'labelright', 'labeltop', 'labelrotation', 'grid_agg_filter', 'grid_alpha', 'grid_animated', 'grid_antialiased', 'grid_clip_box', 'grid_clip_on', 'grid_clip_path', 'grid_color', 'grid_dash_capstyle', 'grid_dash_joinstyle', 'grid_dashes', 'grid_data', 'grid_drawstyle', 'g

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
if activate_dtale:
    d = dtale.show(df[columns])
    d.open_browser()

In [7]:
if dim_reduction is not None:

    sns.set(rc={'figure.figsize':(11,8)})
    sns.set(font_scale=1.2)
    plt.clf()
    features = df[c_ssrs_intake]
    features = features.dropna(how= 'any', axis=0)

    # Instantiate PCA and fit_transform your features
    model = dim_reduction(n_components=2)
    model_features = model.fit_transform(features)

    # Concatenate your PCA features with your target variable
    model_df = pd.DataFrame(model_features, columns=['var1', 'var2'])

    # Plot the output using Seaborn
    sns.scatterplot(x='var1', y='var2', data=model_df).figure.savefig(fr"../cluster_vis/{name}_vis.png") 
